# TP YAMS - EL KETTANEH Joseph - M2 SD
##### Yams is a well-known dice game in which the goal is to achieve a high score by rolling certain dice combinations

# Computing the optimal policy

## In The following code we have the functions used to compute the optimal policy

- get_states(nb_dices,nb_faces):
    - Returns the set of states S and the probability of each state for a defined number of dices anf faces
- get_actions_from_state(my_States,nb_faces)
    - Returns the set of actions that can be taken for a defined state
- get_states_from_action(my_actions,nb_dices)
    - Returns the set of states of a taken Action and the probability of each state
- get_state_index(my_state,my_states,nb_faces)
    - Returns the index of state of the set of states
- set_last_rewards(s,nb_faces) 
    - Returns the game score as reward
- optimal_policy(nb_dices, nb_faces)
    - Returns the value function v ∈ R|S|×3 and the optimal policy as the best action a∗(s) for each state, π ∈ N|S|×6×3

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import random
import itertools
from itertools import product

#function to roll the dice
def roll():
    dices_init = np.zeros(NB_DICES,dtype='int')
    faces_init = np.zeros(NB_FACES,dtype='int')
    for i in range(NB_DICES):
        face = random.randint(1,NB_FACES)
        faces_init[face- 1] += 1
    return faces_init

def get_states(nb_dices,nb_faces):
    S = []
    for it in (itertools.product(range(nb_faces) , repeat = nb_dices)):
        s = np.zeros((nb_faces), dtype='int')

        for f in range(nb_faces):
            s[f] = (np.array(it) == f).sum()
            if s.sum() == nb_faces:
                break
        S.append(s)

    S,counts = np.unique(S, axis=0, return_counts=True)
    return list(S), counts/counts.sum()

def get_actions_from_state(my_States,nb_faces):
    #nb_actions = np.prod(my_States+1)

    nb_actions=np.prod(my_States+1)
    #print(nb_actions)
    Actions = np.zeros((nb_actions,nb_faces),dtype='int')
    l = []
    for f in range(nb_faces):
        l.append(list(np.arange(my_States[f]+1)))
    k = 0
    # then generate all the possible actions
    for i in itertools.product(*l):
        Actions[k,:] = np.array(i)
        k = k + 1
    return Actions

def get_states_from_action(my_actions,nb_dices):
    r = nb_dices - my_actions.sum()
    #print(my_actions.sum())
    ls,lp = get_states(r,NB_FACES)
    S = ls + my_actions
    return S ,lp

def get_state_index(my_state,my_states,nb_faces):
    return np.argwhere((my_state == my_states).sum(axis=1) == nb_faces)[0][0]

def set_last_rewards(s,nb_faces):
    a = np.arange(1,nb_faces+1)
    r = 0
    if list(s).count(1) == 3:
        num = []
        for i in range(len(s)):
            if s[i] == 1:
                num.append(i)
            if num == [0,1,2,3] or num == [1,2,3,4] or num == [2,3,4,5]:
                r = 25
            
    else:
        if list(s).count(1) == 5:
            num = []
            for i in range(len(s)):
                   if s[i] == 1:
                           num.append(i)
            if num == [1,2,3,4,5] or num == [0,1,2,3,4]:
                r = 40
            
        else:
            for i in s: 
                if i == 3:
                    for j in s[i:]: 
                        if j == 2: 
                            r = 25 #full
                        else: 
                            r = np.sum(s*a) #brelan
                if i == 4:
                    r = np.sum(s*a) #carré
                if i == 5: 
                    r = 50
    if r == 0:
        r = np.sum(a*s)
        
    return r

def calcul_R_A(S,R0, nb_faces,nb_dices):
    A_optimal = []
    R = []
    for s in S:
        A = get_actions_from_state(s,nb_faces)
        ra = 0 #reward moyenne de cette action
        Ra = []
        for a in A:
            Sa, Pa = get_states_from_action(a,nb_dices)
            ra = 0
            for i,sa in enumerate(Sa):
                ra += R0[get_state_index(sa,S,nb_faces)]*Pa[i]
            Ra.append(ra)
        index = Ra.index(max(Ra))
        R.append(max(Ra))
        A_optimal.append(np.sum(A[index]))
    return R, A_optimal

def optimal_policy1(nb_dices, nb_faces): 
    States, Probs = get_states(nb_dices,nb_faces)
    Rewards = []
    for state in States: 
        Rewards.append(set_last_rewards(state, nb_faces))

    Value_func = np.zeros(len(States))
    Action1_optimal = []
    Value_func_1 = []
    for state in States:
        Actions = get_actions_from_state(state,nb_faces)
        Q_avg = []
        for action in Actions:
            action_states, action_probs = get_states_from_action(action,nb_dices)
            q_avg = 0
            for i,action_state in enumerate(action_states):
                q_avg += Rewards[get_state_index(action_state,States,nb_faces)]*action_probs[i]
            Q_avg.append(q_avg)
        index_max_q = Q_avg.index(max(Q_avg))
        Action1_optimal.append(np.sum(Actions[index_max_q]))
        Value_func_1.append(max(Q_avg))
        
    Value_func_2 = []
    Action2_optimal = []
    for state in States:
        Actions = get_actions_from_state(state,nb_faces)
        Q_avg = []
        for action in Actions:
            action_states, action_probs = get_states_from_action(action,nb_dices)
            q_avg = 0
            for i,action_state in enumerate(action_states):
                q_avg += Value_func_1[get_state_index(action_state,States,nb_faces)]*action_probs[i]
            Q_avg.append(q_avg)
        index_max_q = Q_avg.index(max(Q_avg))
        Value_func_2.append(max(Q_avg))
        Action2_optimal.append(np.sum(Actions[index_max_q]))
    Value_func_f = np.zeros((len(States), 3))
    #Value_func_f[:,0] = States
    Value_func_f[:,0] = Rewards
    Value_func_f[:,1] = Value_func_1
    Value_func_f[:,2] = Value_func_2
    Optimal_Actions = np.zeros((len(States),2))
    #Actions[:,1] = States
    Optimal_Actions[:,0] = Action1_optimal
    Optimal_Actions[:,1] = Action2_optimal
    return Value_func_f,Optimal_Actions, States


In [38]:
#set up environment Variables
NB_DICES = 5
NB_FACES = 6
# Compute the optimal Policy
Value_func_f, Optimal_Actions, States = optimal_policy1(NB_DICES,NB_FACES)

### For Each state the optimal policy is computed and the results in the dataframe as follows:

In [66]:
import pandas as pd
import numpy as np

columns = ["Face1","Face2","Face3","Face4","Face5","Face6"]
df_states = pd.DataFrame(States, columns=columns)

columns = ["V0","V1","V2"]
df_value_func_f = pd.DataFrame(Value_func_f, columns=columns)

columns = ["A1","A2"]
df_Optimal_Actions = pd.DataFrame(Optimal_Actions, columns=columns)
df_optimal_pol = pd.concat([df_states, df_value_func_f,df_Optimal_Actions], axis=1)
df_optimal_pol_f = df_optimal_pol.loc[:, ["Face1","Face2","Face3","Face4","Face5","Face6", "V0", "A1", "V1", "A2","V2"]]
df_optimal_pol_f

,Face1,Face2,Face3,Face4,Face5,Face6,V0,A1,V1,A2,V2
0,0,0,0,0,0,5,50.0,5.0,50.000000,5.0,50.000000
1,0,0,0,0,1,4,29.0,4.0,30.833333,4.0,34.027778
2,0,0,0,0,2,3,28.0,5.0,28.000000,3.0,28.018519
3,0,0,0,0,3,2,25.0,4.0,25.166667,4.0,25.777778
4,0,0,0,0,4,1,26.0,4.0,27.666667,4.0,31.388889
...,...,...,...,...,...,...,...,...,...,...,...
247,4,0,0,0,1,0,9.0,1.0,19.843364,4.0,24.273748
248,4,0,0,1,0,0,8.0,1.0,18.861111,4.0,24.273748
249,4,0,1,0,0,0,7.0,0.0,18.245242,4.0,24.273748
250,4,1,0,0,0,0,6.0,0.0,18.245242,4.0,24.273748


   The concept is that during each turn, a player has three chances to roll the highest possible amount on 5 6-sided dice. All dice are rolled at first <b>A0</b>, and then the player gets to choose which dice to hold and which to re-roll on subsequent rolls <b>A1</b>. The final roll is scored according to the following table <b>A2</b>

The state space S is the set of distinct sorted rolls of 5 6-sided dice (5d6). There are 65=7776 possible 5d6 rolls, but resulting in only 252 distinct sorted dice patterns, so |S|=252. 

- V0 is the Initial Returns
- A1 is the 2nd Action taken after:
    - 5: keep 5 dices
    - 4: keep 4 dices and reroll 1 dice
    - 3: keep 3 dices and reroll 2 dices
    - 2: keep 3 dices and reroll 2 dices
    - 1: keep 1 dices and reroll 4 dices
    - 0: reroll all dices
- V1 is the rewars returnes of Action1
- A2 is the 3rd action taken after V1
- V2 is the final reward or gain

So each row in the dataframe represents the Optimal policy taken for each state the maximizes the Rewards
